In [1]:
import logging
import coloredlogs
import requests
from time import sleep

from psycopg2.extensions import cursor
from psycopg2.extensions import connection
import pandas as pd

In [2]:
logging.basicConfig()
# logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [144]:
import config
from imp import reload
reload(config)
from config import config
import psycopg2

# connect to the db
my_connection = psycopg2.connect(**config['postgres'])
my_cursor = my_connection.cursor()
logging.info('Connected to database.')

2021-05-27 15:45:02 congratulations root[12723] INFO Connected to database.


In [147]:
reload(ooni_utils)
import src.ooni.utils as ooni_utils

In [18]:
def rollback():
    my_cursor.execute("ROLLBACK")
    my_connection.commit()

# Visualization!

just connect to database to run cells below.

In [11]:
import json

load_json_file = lambda fn: json.load(open(fn))

sampleData = load_json_file('../giojs/examples/data/sampleData.json')

In [12]:
sampleData

[{'e': 'PE', 'i': 'PG', 'v': 2935070},
 {'e': 'PE', 'i': 'NL', 'v': 6408452},
 {'e': 'PE', 'i': 'TO', 'v': 1912811},
 {'e': 'PE', 'i': 'AE', 'v': 6119993},
 {'e': 'PE', 'i': 'GG', 'v': 6793973},
 {'e': 'BF', 'i': 'UG', 'v': 7672134},
 {'e': 'BF', 'i': 'HR', 'v': 5690279},
 {'e': 'BF', 'i': 'FK', 'v': 1069550},
 {'e': 'BF', 'i': 'PY', 'v': 667732},
 {'e': 'BF', 'i': 'LT', 'v': 1469193},
 {'e': 'BF', 'i': 'SB', 'v': 4240389},
 {'e': 'BF', 'i': 'IS', 'v': 2743953},
 {'e': 'BF', 'i': 'IN', 'v': 5220844},
 {'e': 'BF', 'i': 'MV', 'v': 1816543},
 {'e': 'BF', 'i': 'ZM', 'v': 3438115},
 {'e': 'BF', 'i': 'PN', 'v': 3802433},
 {'e': 'BF', 'i': 'AM', 'v': 1491657},
 {'e': 'FR', 'i': 'BG', 'v': 1159615},
 {'e': 'FR', 'i': 'PR', 'v': 7468378},
 {'e': 'FR', 'i': 'SD', 'v': 210974},
 {'e': 'FR', 'i': 'KP', 'v': 7829956},
 {'e': 'FR', 'i': 'SH', 'v': 3559675},
 {'e': 'FR', 'i': 'IE', 'v': 2282346},
 {'e': 'FR', 'i': 'AD', 'v': 7504218},
 {'e': 'FR', 'i': 'AQ', 'v': 2924604},
 {'e': 'FR', 'i': 'GL', 'v'

import country
export country
value

e -> i


countries are CC2s (perfect)

# Get directed data from the db

In [78]:
# rollback()

import pandas as pd

def unique_pairs (target_attribute:str, confirmed_only=True) -> pd.DataFrame:
    '''
    These are unique inputs that are blocked.
    
    We don't want every blocking example - if a site appears once ore more, that's enough. It'll show up here.
    '''
    my_cursor.execute(f'''
    SELECT DISTINCT input_url, probe_alpha2, {target_attribute} 
    from ooni_web_connectivity_test
    {"where confirmed=True" if confirmed_only else ""}
    ''')
    pairs = my_cursor.fetchall()
    return pd.DataFrame(pairs, columns=['input', 'e', 'i']).dropna()


In [36]:
# dummy data
OONIWebConnectivityTest(
    'example',
    Alpha2('US'),
    'example',
    False,
    False,
    'example',
    Alpha2('US'),
    Alpha2('US'),
    pd.Timestamp('2000-01-01 21:41:37+00:00')
).create_table(my_cursor, my_connection)

In [81]:
def write_to_db (cur: cursor, conn: connection, connectivity_tests: List[OONIWebConnectivityTest]) -> None:
    for t in connectivity_tests:
        t.write_to_db(cur, conn, commit=False)
    conn.commit()

In [112]:
# rollback()
from funcy import flatten

def create_weights (**kwargs) -> dict:
    # get data
    pairs_tld = unique_pairs('tld_jurisdiction_alpha2', **kwargs)
#     pairs_ip = unique_pairs('input_ip_alpha2', **kwargs)
    attacks = pd.concat([pairs_tld, pairs_ip])
    # find discrete lines of attack
    lines_of_attack = set([(v[0], v[1]) for v in attacks[['e', 'i']].values])
    # find safe/approved CC2s
    cc2_in_our =  set(flatten(attacks[['e', 'i']].values.tolist()))
    cc2_in_their =  set(flatten([(s['e'], s['i']) for s in sampleData]))
    possibly_unsafe_cc2 = cc2_in_our.difference(cc2_in_their)
    # construct weights
    weights = []
    for line in lines_of_attack:
        e, i = line
        # TODO - hack
        if e not in possibly_unsafe_cc2 and i not in possibly_unsafe_cc2 and e != i:
            num_attacks = len(attacks[(attacks['e']==e) & (attacks['i']==i)])
            weight = num_attacks*10000
            weights.append({
                'e': e,
                'i': i,
                'v': weight
            })
    return weights

In [38]:
# my_connection.commit()

In [113]:
weights_confirmed = create_weights(confirmed_only=True)
weights_all = create_weights(confirmed_only=False)

weights = {
    'dataSetKeys': ['confirmed', 'all'],
    'initDataSetKey': 'confirmed',
    'confirmed': weights_confirmed,
    'all': weights_all
}


In [156]:
ms[0]

{'anomaly': True,
 'confirmed': False,
 'failure': False,
 'input': 'http://cnpolitics.org/',
 'measurement_start_time': '2021-05-27T23:18:57Z',
 'measurement_url': 'https://ams-pg.ooni.org/api/v1/raw_measurement?report_id=20210527T224247Z_webconnectivity_CN_4837_n1_rBUK7eAiUOZaGFvN&input=http%3A%2F%2Fcnpolitics.org%2F',
 'probe_asn': 'AS4837',
 'probe_cc': 'CN',
 'report_id': '20210527T224247Z_webconnectivity_CN_4837_n1_rBUK7eAiUOZaGFvN',
 'scores': {'analysis': {'blocking_type': 'http-failure'},
  'blocking_country': 0.0,
  'blocking_general': 1.0,
  'blocking_global': 0.0,
  'blocking_isp': 0.0,
  'blocking_local': 0.0},
 'test_name': 'web_connectivity'}

In [124]:
json.dump(weights, open('visualizations/globe/data/attacks.json', 'w'))

# Some basic questions

how many 'bullets' do we see going by?

In [118]:
pairs_tld = unique_pairs('tld_jurisdiction_alpha2', confirmed_only=True)
pairs_tld
# about 418?

,input,e,i
0,http://rapidgator.net/,RU,US
1,http://www.umc.org/,IR,US
2,http://www.kasparov.ru/,RU,RU
3,http://www.metal-archives.com/,IR,US
4,http://www.budweiser.com/,IR,US
...,...,...,...
413,http://www.proxyweb.net/,IR,US
414,http://www.cbsnews.com/,IR,US
415,http://speed-vpn.com/,IR,US
416,http://www.hustler.com/,IR,US


about 418/day?

by destination, who's receiving them?

In [123]:
pairs_tld.groupby('i').count().sort_values('e', ascending=False)

,input,e
i,,
US,360,360
RU,17,17
GB,5,5
UA,4,4
TV,4,4
FR,3,3
CA,2,2
CC,2,2
CN,2,2


maybe just a big datacenter there...? load balancer? unclear.
unclear what to make of input IP in general...